In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import sklearn
from sklearn import metrics, model_selection, tree
from sklearn.ensemble import GradientBoostingClassifier
import joblib

In [2]:
# Train data
X_train = pd.read_csv('x_train_smote_undersample.csv').to_numpy()
y_train = pd.read_csv('y_train_smote_undersample.csv')
y_train = y_train['LABEL']

# Test data
X_test = pd.read_csv('x_test_smote_undersample.csv').to_numpy()
y_test = pd.read_csv('y_test_smote_undersample.csv')
y_test = y_test['LABEL']

# Validation data
X_validation = pd.read_csv('x_validation_smote_undersample.csv').to_numpy()
y_validation = pd.read_csv('y_validation_smote_undersample.csv')
y_validation = y_validation['LABEL']

In [3]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=7)

In [4]:
clf.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=7)

In [5]:
clf.score(X_test, y_test)

0.8854319869588843

In [6]:
pred = clf.predict(X_test)

In [7]:
print(metrics.accuracy_score(y_test, pred))
print('Matriz de confusion', metrics.confusion_matrix(y_test, pred))
print(metrics.classification_report(y_test, pred, target_names=['0', '1', '2', '3']))

0.8854319869588843
Matriz de confusion [[276634    139   4740  15116]
 [    23 112711      0     85]
 [  5858      0  60290  37292]
 [     0      0      0  39212]]
              precision    recall  f1-score   support

           0       0.98      0.93      0.96    296629
           1       1.00      1.00      1.00    112819
           2       0.93      0.58      0.72    103440
           3       0.43      1.00      0.60     39212

    accuracy                           0.89    552100
   macro avg       0.83      0.88      0.82    552100
weighted avg       0.93      0.89      0.89    552100



In [8]:
# metrics.plot_roc_curve(clf, X_test, y_test)
# plt.show()

In [9]:
joblib.dump(clf, 'modelo1_v2.h5')

['modelo1_v2.h5']